In [ ]:
! ls /home/vmagent/app/RedPajama-Data-1T-Sample

In [2]:
import argparse
import os
import sys

cur_path = os.path.join(os.path.abspath(''), "../../../..")
print(cur_path)
sys.path.append(cur_path)
from modelzoo.transformers.data_processing.slimpajama.dedup import (
    dedup_train,
    generate_connected_components,
    generate_duplicate_pairs,
    generate_duplicates_dict,
    to_hash,
)
from modelzoo.transformers.data_processing.slimpajama.preprocessing import (
    datasets,
    filter,
    normalize_text,
    shuffle_holdout,
)

/home/vmagent/app/workspace/modelzoo/modelzoo/transformers/data_processing/slimpajama/../../../..


In [3]:
from pyrecdp.core.utils import Timer

JAVA_HOME is not set, use default value of /usr/lib/jvm/java-8-openjdk-amd64/


/usr/local/lib/python3.10/dist-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [9]:
ds_names = [
    "arxiv",
    "stackexchange",
    "book",
    "wikipedia",
    "github",
    "c4",
    #"common_crawl",
]
cc_years = [""]
n_documents = [
    1558306,
    29825086,
    205744,
    29834171,
    28793312,
    364868892,
    81085420,
    90850492,
    98878523,
    94058868,
    111402716,
]

In [8]:
# norm text
ds_dirs = ds_names.copy()
ds_dirs.remove("common_crawl")
for cc in cc_years:
    ds_dirs.append("common_crawl/" + cc)

input_dir = "/home/vmagent/app/RedPajama-Data-1T-Sample/"
red_pj_norm = os.path.join(input_dir, "RedPajama_norm")
with Timer(f"norm all"):
    for dataset in ds_dirs:
        with Timer(f"norm {dataset}"):
            norm_args = argparse.Namespace()
            norm_args.data_dir = os.path.join(input_dir, dataset)
            norm_args.target_dir = os.path.join(red_pj_norm, dataset)
            norm_args.zst = "common_crawl" in dataset
            norm_args.idx = -1
            normalize_text.normalize_text(norm_args)

norm all started ...
norm arxiv started ...
resetting to 1 for number of processes


Parsed 0 input files. Files written : 0it [00:21, ?it/s]                                                                                              | 0/1 [00:00<?, ?it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:21<00:00, 21.52s/it]

  norm arxiv took 21.552981715649366 sec
norm stackexchange started ...
resetting to 1 for number of processes



Parsed 0 input files. Files written : 0it [00:23, ?it/s]                                                                                              | 0/1 [00:00<?, ?it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:23<00:00, 23.87s/it]

  norm stackexchange took 23.89708335418254 sec
norm book started ...
resetting to 1 for number of processes


...


Parsed 0 input files. Files written : 0it [01:19, ?it/s]                                                                                              | 0/1 [00:00<?, ?it/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:19<00:00, 79.13s/it]

  norm github took 79.16304822638631 sec
norm c4 started ...
resetting to 1 for number of processes



Parsed 0 input files. Files written : 0it [04:19, ?it/s]                                                                                              | 0/1 [00:00<?, ?it/s]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [04:19<00:00, 259.90s/it]

  norm c4 took 259.93023099098355 sec
norm common_crawl/2019-30 started ...
  norm common_crawl/2019-30 took 0.0006070993840694427 sec
norm all took 455.3269019080326 sec


FileNotFoundError: [Errno 2] No such file or directory: '/home/vmagent/app/RedPajama-Data-1T-Sample/common_crawl/2019-30'

In [10]:
# filter docs
with Timer(f"filter all"):
    short_docs = os.path.join(red_pj_norm, "red_pj_filter.pickle")
    filter_args = argparse.Namespace()
    filter_args.input_dir = red_pj_norm
    filter_args.output_file = short_docs
    filter_args.n_docs = sum(n_documents)
    filter_args.dataset_name = "all"
    filter_args.threshold = 200
    filter.filter_dataset(filter_args)

filter all started ...
Processed 0.0%.  0.00038123130798339844
Processed 1.0736969133779877e-06%.  0.0015087127685546875
Processed 2.1473938267559754e-06%.  0.0016374588012695312
Processed 3.2210907401339627e-06%.  0.0017664432525634766
Processed 4.294787653511951e-06%.  0.001888275146484375
Processed 5.368484566889938e-06%.  0.0020225048065185547
Processed 6.4421814802679255e-06%.  0.002138376235961914
Processed 7.515878393645914e-06%.  0.0022521018981933594
Processed 8.589575307023902e-06%.  0.00321197509765625
Processed 9.663272220401889e-06%.  0.12241649627685547
Processed 1.0736969133779876e-05%.  0.16431498527526855
Processed 1.1810666047157864e-05%.  0.1932506561279297
Processed 1.2884362960535851e-05%.  0.24463844299316406
Processed 1.3958059873913838e-05%.  0.2971839904785156
Processed 1.5031756787291827e-05%.  0.36751365661621094
Processed 1.6105453700669816e-05%.  0.4253532886505127
Processed 1.7179150614047804e-05%.  0.494734525680542
Processed 1.825284752742579e-05%.  0.51

In [11]:
# generate minhash
with Timer(f"minhash all"):
    for idx, dataset in enumerate(ds_dirs):
        with Timer(f"minhash {dataset}"):
            hash_args = argparse.Namespace()
            hash_args.dataset_name = (
                "common_crawl" if "common_crawl" in dataset else dataset
            )
            hash_args.input_dir = os.path.join(red_pj_norm, dataset)
            hash_args.output_dir = os.path.join(red_pj_norm, dataset)
            hash_args.n_docs = n_documents[idx]
            hash_args.iter = 0
            hash_args.index_start = 0
            hash_args.index_end = None
            hash_args.w = 13
            hash_args.k = 10000
            to_hash.generate_hashes(hash_args)


minhash all started ...
minhash arxiv started ...


  0%|                                                                                                                                                | 0/96 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  3.33it/s]
  1%|█▍                                                                                                                                  | 1/96 [11:38<18:25:28, 698.19s/it]

  minhash arxiv took 699.0779332155362 sec
minhash stackexchange started ...



  0%|                                                                                                                                                | 0/96 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  2.78it/s]
  1%|█▍                                                                                                                                  | 1/96 [09:36<15:13:01, 576.64s/it]


  minhash stackexchange took 577.6513189533725 sec
minhash book started ...


...

  0%|                                                                                                                                                | 0/96 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:03,  3.33s/it]
  1%|█▎                                                                                                                              | 1/96 [2:06:13<199:52:09, 7573.99s/it]


  minhash c4 took 7575.667032993399 sec
minhash common_crawl/2019-30 started ...


  0%|                                                                                                                                                                                                                                                                                          | 0/96 [00:00<?, ?it/s]
0it [00:00, ?it/s]
  0%|                                                                                                                                                                                                                                                                                          | 0/96 [00:00<?, ?it/s]

  minhash common_crawl/2019-30 took 1.9990124739706516 sec
minhash common_crawl/2020-05 started ...
  minhash common_crawl/2020-05 took 0.0005881106480956078 sec
minhash all took 11757.24178331811 sec


FileNotFoundError: [Errno 2] No such file or directory: '/home/vmagent/app/RedPajama-Data-1T-Sample/RedPajama_norm/common_crawl/2020-05/minhash_nfc'

In [12]:
# generate duplicates
with Timer(f"generate pairs all"):
    dup_dir = os.path.join(red_pj_norm, "dup")
    os.makedirs(dup_dir, exist_ok=True)
    dup_pairs_args = argparse.Namespace()
    dup_pairs_args.input_dir = red_pj_norm
    dup_pairs_args.out_file = os.path.join(dup_dir, "duplicate_pairs.txt")
    dup_pairs_args.range = 13
    dup_pairs_args.bands = 9
    dup_pairs_args.processes = 45
    generate_duplicate_pairs.generate_pairs(dup_pairs_args)


generate pairs all started ...
0: Processed 0.0%. 0.0008964538574218751: Processed 0.0%.
 2: Processed 0.0%.0.0008795261383056641
 0.00050425529479980473: Processed 0.0%.
 0.0008826255798339844
4: Processed 0.0%.5: Processed 0.0%.  0.00087642669677734380.0004329681396484375

6: Processed 0.0%. 0.00085902214050292977: Processed 0.0%.
 0.0008318424224853516
8: Processed 0.0%. 0.0008518695831298828
0: Processed 0.010736969133779875%. 4.4366114139556885
1: Processed 0.010736969133779875%. 4.548372030258179
2: Processed 0.010736969133779875%. 4.588858366012573
8: Processed 0.010736969133779875%. 4.43910026550293
5: Processed 0.010736969133779875%. 4.54279088973999
6: Processed 0.010736969133779875%. 4.525859832763672
3: Processed 0.010736969133779875%. 4.623370409011841
4: Processed 0.010736969133779875%. 4.603491544723511
7: Processed 0.010736969133779875%. 4.567554473876953
0: Processed 0.02147393826755975%. 8.608381271362305
1: Processed 0.02147393826755975%. 8.980336427688599
8: Process

In [13]:
with Timer(f"generate_connected_components all"):
    dup_connected_args = argparse.Namespace()
    dup_connected_args.input_dir = dup_dir
    dup_connected_args.out_file = os.path.join(
        dup_dir, "connected_components.pickle"
    )
    generate_connected_components.generate_connected_components_mp(
        dup_connected_args
    )



generate_connected_components all started ...
Started graph building


1660it [00:00, 290118.11it/s]
1664it [00:00, 350596.37it/s]
1683it [00:00, 240806.91it/s]
1606it [00:00, 425443.83it/s]
1629it [00:00, 464734.13it/s]
1624it [00:00, 358351.73it/s]
1607it [00:00, 319533.83it/s]
1683it [00:00, 376641.43it/s]
1646it [00:00, 412414.84it/s]


length of the set of duplicates: 6007 0.08033490180969238


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6007/6007 [00:00<00:00, 454081.82it/s]

number of connected components: 1418 0.09987139701843262
Graph generated duplicates list!!! 0.10342788696289062
generate_connected_components all took 0.10580181702971458 sec


In [14]:
with Timer(f"generate_duplicates_dict all"):
    dup_docs = os.path.join(dup_dir, "duplicates.pickle")
    dup_dict_args = argparse.Namespace()
    dup_dict_args.input_file = os.path.join(
        dup_dir, "connected_components.pickle"
    )
    dup_dict_args.out_file = dup_docs
    generate_duplicates_dict.generate_duplicates(dup_dict_args)

generate_duplicates_dict all started ...
Processing duplicates!!!


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1418/1418 [00:00<00:00, 115931.60it/s]

number of duplicate documents that will be removed: 4222
generate_duplicates_dict all took 0.025027991272509098 sec
